<a href="https://colab.research.google.com/github/yufengg/advent-of-code-2024/blob/master/AoC_2024_day_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: python code download advent of code file

import requests
import os
from datetime import datetime

def download_aoc_input(year, day, session_cookie):
  """Downloads the input file for a specific Advent of Code day.

  Args:
    year: The year of the challenge (e.g., 2023).
    day: The day of the challenge (e.g., 1).
    session_cookie: Your session cookie from the Advent of Code website.
  """

  url = f"https://adventofcode.com/{year}/day/{day}/input"
  headers = {
      "User-Agent": "github.com/yourusername/yourrepo (or your email)",  # Replace with your info
      "Cookie": f"session={session_cookie}"
  }

  try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Create a directory for the year if it doesn't exist
    os.makedirs(str(year), exist_ok=True)

    filename = os.path.join(str(year), f"day{day}.txt")
    with open(filename, "w") as file:
      file.write(response.text)
    print(f"Successfully downloaded input for year {year}, day {day} to {filename}")
    return filename

  except requests.exceptions.RequestException as e:
    print(f"Error downloading input: {e}")
  except Exception as e:
    print(f"An unexpected error occurred: {e}")


# Example usage:  Replace with your actual values
year = 2024  #@param {type:"integer"}
day = 10 #@param {type:"integer"}
session_cookie = "53616c7465645f5fd6809f86933785dcec3c4f79b521758a796b0808898e32ff9a09d06dd15282b2a68340b7bf970ab66894fa0bf9271caa521bc5d5fc8df3b9" #@param {type:"string"}

input_filename = download_aoc_input(year, day, session_cookie)

Successfully downloaded input for year 2024, day 10 to 2024/day10.txt


In [41]:
import jax
import jax.numpy as jnp
import numpy as np
from collections import defaultdict


Need a matrix representation of the grid, probably a class.

for each location, determine which directions are valid (up, down, left, right), where the next number is exactly n+1. Track the locations of trailheads (n=0) and trailends (n=9) as 2 sets of coordinates.

This can be built as a dict mapping coordinate (x,y) to list[directions].

Then for each trailhead, following the directions recursively until we hit a trailend, returning its _location_, or a location with no direction, represented as []. Then store found trailends in a set() to remove dups. Count up the trailends.

In [4]:

def read_input(filename, parser=None):
  with open(filename, 'r') as file:
    lines = file.read().splitlines()
    if parser:
      lines = list(map(parser, lines))
  return lines


In [19]:
input_filename = f'{year}/day{day}.txt'
def parser(line):
  return [int(n) for n in line]
  # return tuple(map(int, line.split()))



In [9]:
'89010123'.split()

['89010123']

# Test data

In [14]:
test_data = '''89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
'''
test_file = f'day{day}_test.txt'
with open(test_file, 'w') as file:
  lines = file.writelines(test_data)


In [20]:
lines = read_input(test_file, parser)


In [21]:
lines

[[8, 9, 0, 1, 0, 1, 2, 3],
 [7, 8, 1, 2, 1, 8, 7, 4],
 [8, 7, 4, 3, 0, 9, 6, 5],
 [9, 6, 5, 4, 9, 8, 7, 4],
 [4, 5, 6, 7, 8, 9, 0, 3],
 [3, 2, 0, 1, 9, 0, 1, 2],
 [0, 1, 3, 2, 9, 8, 0, 1],
 [1, 0, 4, 5, 6, 7, 3, 2]]

In [30]:
lines[0][3]

1

In [35]:
'\n'.join([7, 8, 1, 2, 1, 8, 7, 4])

TypeError: sequence item 2: expected str instance, int found

In [40]:
# '\n'.join(''.join(lines))

'\n'.join([''.join(map(str,l)) for l in lines])

'89010123\n78121874\n87430965\n96549874\n45678903\n32019012\n01329801\n10456732'

In [ ]:
#location of the guard is?
for i in range(len(lines)):
  for j in range(len(lines[0])):
    if lines[i][j] == '^':
      guard_pos = (i,j)

In [ ]:
guard_pos

(6, 4)

In [100]:
class Grid:
  def __init__(self, lines):
    self.lines = lines
    self.height = len(lines)
    self.width = len(lines[0])
    self.trailheads = set()
    self.trailends = set()
    self.direction_dict = defaultdict(list)


  def __str__(self):
    # return '\n'.join(self.lines)
    return '\n'.join([''.join(map(str,l)) for l in lines])


  def find_guard(self):
    for i in range(self.height):
      for j in range(self.width):
        if self.lines[i][j] == '^':
          return np.array([i,j])

  def print_grid(self):
    for line in self.lines:
      print(line)

  def direction_to_delta(self, direction):
    if direction == 'N':
      return np.array([-1,0])
    elif direction == 'S':
      return np.array([1,0])
    elif direction == 'E':
      return np.array([0,1])
    elif direction == 'W':
      return np.array([0,-1])

  # just checks if it's on the grid
  def validate_step(self, coordinate):
    if coordinate[0] < 0 or coordinate[0] >= self.height:
      return False
    if coordinate[1] < 0 or coordinate[1] >= self.width:
      return False
    return True

  # go through the matrix and mark each position with all valid direcitons the trail can travel
  def step1_annotate(self):
    for i in range(self.height):
      for j in range(self.width):
        for direction in ['N', 'S', 'E', 'W']:
          value = self.lines[i][j]
          potential_step = np.array([i,j]) + self.direction_to_delta(direction)
          if not self.validate_step(potential_step):
            continue
          if self.lines[potential_step[0]][potential_step[1]] == value + 1:
            self.direction_dict[(i,j)].append(direction)

        if value == 0:
          self.trailheads.add((i,j))
        if value == 9:
          self.trailends.add((i,j))

  def trace_trail(self, trailhead_loc):
    i,j = trailhead_loc
    ends = set()
    if tuple((i,j)) in self.trailends:
      # locs = set()
      ends.add(trailhead_loc)
      return ends

    for direction in self.direction_dict[trailhead_loc]:
      step = np.array([i,j]) + self.direction_to_delta(direction)
      trailend = self.trace_trail((step[0], step[1]))
      if trailend:
        ends.update(trailend)

    if ends:
      return ends

    return False



  def count_trailheads(self):
    num_ends = 0
    for trailhead in self.trailheads:
      ends = self.trace_trail(trailhead)
      print(trailhead, ends)
      if ends:
        num_ends += len(ends)
    return num_ends

In [101]:
g = Grid(lines)

In [102]:
g.step1_annotate()

In [103]:
g.count_trailheads()

(2, 4) {(0, 1), (4, 5), (3, 4), (5, 4), (3, 0)}
(5, 5) {(4, 5), (2, 5), (3, 4)}
(0, 4) {(0, 1), (3, 4), (5, 4), (3, 0), (4, 5), (2, 5)}
(7, 1) {(0, 1), (4, 5), (3, 4), (5, 4), (3, 0)}
(4, 6) {(4, 5), (2, 5), (3, 4)}
(0, 2) {(0, 1), (3, 4), (5, 4), (3, 0), (4, 5)}
(6, 0) {(0, 1), (4, 5), (3, 4), (5, 4), (3, 0)}
(6, 6) {(4, 5), (3, 4), (2, 5)}
(5, 2) {(6, 4)}


36

array([4, 4])

# Part 1

In [107]:
lines = read_input(input_filename, parser)


In [108]:
g = Grid(lines)


In [109]:
g.lines

[[4,
  3,
  0,
  8,
  7,
  6,
  5,
  4,
  7,
  6,
  5,
  4,
  1,
  0,
  1,
  0,
  1,
  2,
  3,
  4,
  9,
  8,
  8,
  9,
  8,
  5,
  4,
  3,
  4,
  2,
  3,
  4,
  5,
  8,
  7,
  6,
  7,
  8,
  9,
  8,
  4,
  3,
  2,
  1,
  2,
  3,
  4,
  5,
  4,
  3,
  8,
  7,
  2,
  1,
  0],
 [5,
  2,
  1,
  9,
  0,
  0,
  2,
  3,
  8,
  9,
  5,
  3,
  2,
  3,
  0,
  9,
  8,
  3,
  5,
  5,
  6,
  7,
  3,
  8,
  7,
  6,
  1,
  2,
  2,
  1,
  0,
  5,
  6,
  9,
  6,
  7,
  8,
  7,
  6,
  3,
  0,
  1,
  0,
  0,
  3,
  6,
  7,
  6,
  7,
  8,
  9,
  6,
  5,
  2,
  3],
 [6,
  5,
  7,
  8,
  1,
  1,
  1,
  0,
  0,
  1,
  6,
  5,
  4,
  3,
  1,
  8,
  7,
  5,
  6,
  7,
  6,
  5,
  4,
  8,
  7,
  6,
  0,
  6,
  7,
  8,
  7,
  6,
  3,
  4,
  5,
  6,
  9,
  8,
  5,
  4,
  3,
  2,
  1,
  8,
  7,
  9,
  8,
  7,
  9,
  2,
  6,
  5,
  6,
  7,
  4],
 [7,
  8,
  6,
  9,
  2,
  2,
  3,
  2,
  1,
  0,
  7,
  8,
  9,
  2,
  2,
  3,
  6,
  4,
  3,
  8,
  9,
  0,
  3,
  9,
  4,
  5,
  6,
  5,
  8,
  9,
  0,
  1,
  2,
  3,
  

In [110]:

g.step1_annotate()

g.count_trailheads()

(12, 4) {(8, 7), (10, 5)}
(6, 18) {(7, 16), (3, 20)}
(36, 53) {(33, 51), (34, 52)}
(21, 16) {(26, 18), (22, 16), (23, 15)}
(26, 30) {(32, 31), (25, 28), (23, 28)}
(1, 40) {(3, 37), (0, 38), (2, 36)}
(28, 3) {(31, 5), (32, 0), (28, 4)}
(42, 2) {(40, 7), (40, 5), (43, 2)}
(48, 36) {(50, 33)}
(53, 32) {(54, 28)}
(11, 5) {(8, 7), (10, 5)}
(48, 54) {(46, 51)}
(54, 49) {(54, 52), (53, 53)}
(43, 12) {(41, 15), (48, 12)}
(15, 5) {(14, 9)}
(41, 24) {(40, 26), (37, 23), (39, 21)}
(46, 20) {(49, 24), (44, 21), (47, 22)}
(18, 1) {(20, 2), (21, 1)}
(21, 0) {(20, 2), (25, 3), (21, 1)}
(33, 20) {(32, 16), (28, 20), (32, 18)}
(32, 30) {(32, 31), (37, 32), (32, 27), (35, 32)}
(10, 27) {(12, 22), (12, 30), (13, 29)}
(18, 10) {(17, 14), (17, 10)}
(33, 29) {(37, 32), (35, 32), (32, 27)}
(36, 46) {(34, 49), (40, 49), (41, 48), (34, 47), (34, 51), (39, 44)}
(30, 39) {(33, 37), (33, 39)}
(10, 36) {(13, 38), (11, 32)}
(42, 41) {(40, 42), (39, 41)}
(52, 42) {(51, 38), (51, 40)}
(8, 48) {(8, 47), (11, 54), (7, 

698

# part 2

this is nice, as we can just stop de-duping all the trailends, and that should be all it takes

In [113]:
class Grid:
  def __init__(self, lines):
    self.lines = lines
    self.height = len(lines)
    self.width = len(lines[0])
    self.trailheads = set()
    self.trailends = set()
    self.direction_dict = defaultdict(list)


  def __str__(self):
    # return '\n'.join(self.lines)
    return '\n'.join([''.join(map(str,l)) for l in lines])


  def find_guard(self):
    for i in range(self.height):
      for j in range(self.width):
        if self.lines[i][j] == '^':
          return np.array([i,j])

  def print_grid(self):
    for line in self.lines:
      print(line)

  def direction_to_delta(self, direction):
    if direction == 'N':
      return np.array([-1,0])
    elif direction == 'S':
      return np.array([1,0])
    elif direction == 'E':
      return np.array([0,1])
    elif direction == 'W':
      return np.array([0,-1])

  # just checks if it's on the grid
  def validate_step(self, coordinate):
    if coordinate[0] < 0 or coordinate[0] >= self.height:
      return False
    if coordinate[1] < 0 or coordinate[1] >= self.width:
      return False
    return True

  # go through the matrix and mark each position with all valid direcitons the trail can travel
  def step1_annotate(self):
    for i in range(self.height):
      for j in range(self.width):
        for direction in ['N', 'S', 'E', 'W']:
          value = self.lines[i][j]
          potential_step = np.array([i,j]) + self.direction_to_delta(direction)
          if not self.validate_step(potential_step):
            continue
          if self.lines[potential_step[0]][potential_step[1]] == value + 1:
            self.direction_dict[(i,j)].append(direction)

        if value == 0:
          self.trailheads.add((i,j))
        if value == 9:
          self.trailends.add((i,j))

  def trace_trail(self, trailhead_loc):
    i,j = trailhead_loc
    ends = list()
    if tuple((i,j)) in self.trailends:
      ends.append(trailhead_loc)
      return ends

    for direction in self.direction_dict[trailhead_loc]:
      step = np.array([i,j]) + self.direction_to_delta(direction)
      trailend = self.trace_trail((step[0], step[1]))
      if trailend:
        ends += trailend

    if len(ends) > 0:
      return ends

    return False



  def count_trailheads(self):
    num_ends = 0
    for trailhead in self.trailheads:
      ends = self.trace_trail(trailhead)
      print(trailhead, ends)
      if ends:
        num_ends += len(ends)
    return num_ends

In [114]:
lines = read_input(test_file, parser)

g = Grid(lines)

In [116]:
lines = read_input(input_filename, parser)

g = Grid(lines)

In [117]:

g.step1_annotate()

g.count_trailheads()

(12, 4) [(8, 7), (10, 5)]
(6, 18) [(3, 20), (3, 20), (3, 20), (7, 16)]
(36, 53) [(34, 52), (33, 51), (34, 52), (33, 51)]
(21, 16) [(23, 15), (23, 15), (22, 16), (26, 18)]
(26, 30) [(32, 31), (32, 31), (23, 28), (25, 28)]
(1, 40) [(3, 37), (0, 38), (3, 37), (2, 36), (2, 36)]
(28, 3) [(31, 5), (32, 0), (28, 4), (31, 5), (31, 5)]
(42, 2) [(43, 2), (40, 7), (40, 5), (43, 2), (40, 7), (40, 5)]
(48, 36) [(50, 33), (50, 33)]
(53, 32) [(54, 28), (54, 28)]
(11, 5) [(8, 7), (10, 5)]
(48, 54) [(46, 51)]
(54, 49) [(54, 52), (53, 53), (53, 53), (54, 52), (53, 53), (53, 53)]
(43, 12) [(41, 15), (41, 15), (41, 15), (41, 15), (48, 12)]
(15, 5) [(14, 9)]
(41, 24) [(40, 26), (37, 23), (37, 23), (39, 21), (37, 23), (39, 21), (37, 23), (39, 21)]
(46, 20) [(44, 21), (44, 21), (44, 21), (49, 24), (49, 24), (47, 22)]
(18, 1) [(21, 1), (20, 2)]
(21, 0) [(20, 2), (21, 1), (25, 3), (20, 2), (21, 1), (25, 3)]
(33, 20) [(28, 20), (32, 18), (32, 16)]
(32, 30) [(32, 31), (32, 31), (32, 27), (35, 32), (35, 32), (37,

1436